# Installing of necessary packages

In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


# Loading of Documents

In [2]:
from langchain_community.document_loaders import Docx2txtLoader
loader = Docx2txtLoader('Refund Policy Document.docx')
data = loader.load()
data

[Document(metadata={'source': 'Refund Policy Document.docx'}, page_content='Refunds and Return Policy\n\n1.\xa0Application for Returns/Refunds\n\n\xa0\n\nSubject to the terms and conditions in this Refunds and Return Policy and the Terms of Service, Buyer may apply for return of the purchased items (“Item”) and/or refund.\n\n\xa0\n\nFor the avoidance of doubt, an application for a refund and/or return may be made whether prior to or after the expiry of the Shopee Guarantee Period as stated in the\xa0Terms of Service. Users may communicate with each other privately to resolve their differences prior, during or after using Shopee Guarantee.\n\n\xa0\n\n2.\xa0Application for the Return of an Item\n\n\xa0\n\nBuyer may only apply for\n\n(i) the refund of the Item where such Item has not been received by Buyer (“Unreceived Items”), or\n\n(ii) the refund and/or return of an Item received by Buyer (“Received Items”) in the following circumstances:\n\na) Buyer received an empty and/or suspicious

In [3]:
data[0].page_content

'Refunds and Return Policy\n\n1.\xa0Application for Returns/Refunds\n\n\xa0\n\nSubject to the terms and conditions in this Refunds and Return Policy and the Terms of Service, Buyer may apply for return of the purchased items (“Item”) and/or refund.\n\n\xa0\n\nFor the avoidance of doubt, an application for a refund and/or return may be made whether prior to or after the expiry of the Shopee Guarantee Period as stated in the\xa0Terms of Service. Users may communicate with each other privately to resolve their differences prior, during or after using Shopee Guarantee.\n\n\xa0\n\n2.\xa0Application for the Return of an Item\n\n\xa0\n\nBuyer may only apply for\n\n(i) the refund of the Item where such Item has not been received by Buyer (“Unreceived Items”), or\n\n(ii) the refund and/or return of an Item received by Buyer (“Received Items”) in the following circumstances:\n\na) Buyer received an empty and/or suspicious parcel;\n\nb) The Item received was incomplete (missing quantity or access

In [4]:
# from langchain_text_splitters import RecursiveCharacterTextSplitter

# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size = 1000,
#     chunk_overlap = 50,
#     length_function = len
#     # Existing args
# )

# documents = text_splitter.split_documents(data)

# documents

In [5]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain.embeddings import HuggingFaceEmbeddings

text_splitter = SemanticChunker(HuggingFaceEmbeddings())
documents = text_splitter.split_documents(data)

C:\Users\Owent\AppData\Local\Temp\ipykernel_29684\1165599744.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  text_splitter = SemanticChunker(HuggingFaceEmbeddings())
C:\Users\Owent\AppData\Local\Temp\ipykernel_29684\1165599744.py:4: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  text_splitter = SemanticChunker(HuggingFaceEmbeddings())
c:\Users\Owent\OneDrive\Desktop\Projects\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter a

In [6]:
print("Number of chunks created: ", len(documents))

for i in range(len(documents)):
    print()
    print(f"CHUNK : {i+1}")
    print(documents[i].page_content)

Number of chunks created:  5

CHUNK : 1
Refunds and Return Policy

1. Application for Returns/Refunds

 

Subject to the terms and conditions in this Refunds and Return Policy and the Terms of Service, Buyer may apply for return of the purchased items (“Item”) and/or refund. For the avoidance of doubt, an application for a refund and/or return may be made whether prior to or after the expiry of the Shopee Guarantee Period as stated in the Terms of Service. Users may communicate with each other privately to resolve their differences prior, during or after using Shopee Guarantee. 2. Application for the Return of an Item

 

Buyer may only apply for

(i) the refund of the Item where such Item has not been received by Buyer (“Unreceived Items”), or

(ii) the refund and/or return of an Item received by Buyer (“Received Items”) in the following circumstances:

a) Buyer received an empty and/or suspicious parcel;

b) The Item received was incomplete (missing quantity or accessories);

c) Sell

# Creating of Chat model

In [7]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Instantiate the embedding model
embedder = HuggingFaceEmbeddings()

# Create the vector store 
vector = FAISS.from_documents(documents, embedder)

C:\Users\Owent\AppData\Local\Temp\ipykernel_29684\2495835858.py:5: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedder = HuggingFaceEmbeddings()


In [ ]:
retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 2})
# retrieved_docs = retriever.invoke("What is the refund policy?")

# print(retrieved_docs[0])

In [9]:
from langchain_community.llms import Ollama

# Define llm
llm = Ollama(model="mistral", temperature=0)

C:\Users\Owent\AppData\Local\Temp\ipykernel_29684\2087247413.py:4: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="mistral", temperature=0)


In [14]:
from langchain.chains import RetrievalQA
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate

prompt = """
1. Use the following pieces of context to answer the question at the end.
2. If you don't know the answer, just say that "I am not too sure. Please refer to https://help.shopee.sg/portal/4/article/77152-Refunds-and-Return-Policy for more details" but don't make up an answer on your own.
3. If the question is completely irrelevant to Shopee's Refund Policy, you should simply reply with "The question is not related to Shopee's Refund Policy", and ask if the user would like to find out about Shopee's Refund Policy. 
4. For example, an irrelevant question like "How is your day?" would warrant the stated response. Additionally, if the question does not mention anything about a refund policy, it should warrant the stated response as well.
5. If you are unsure if the question is relevant or not, you should reply with the above response as well.
6. If the context provided does not help to answer the question, it means that the question is irrelevant, and you should reply with the above response.


Context: {context}

Question: {question}

Helpful Answer:"""


QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt) 

llm_chain = LLMChain(
                  llm=llm, 
                  prompt=QA_CHAIN_PROMPT, 
                  callbacks=None, 
                  verbose=True)

document_prompt = PromptTemplate(
    input_variables=["page_content", "source"],
    template="Context:\ncontent:{page_content}\nsource:{source}",
)

combine_documents_chain = StuffDocumentsChain(
                  llm_chain=llm_chain,
                  document_variable_name="context",
                  document_prompt=document_prompt,
                  callbacks=None,
              )

qa = RetrievalQA(
                  combine_documents_chain=combine_documents_chain,
                  verbose=True,
                  retriever=retriever,
                  return_source_documents=True,
              )


In [17]:
print(qa("How tall are you?")["result"])



> Entering new RetrievalQA chain...


> Entering new LLMChain chain...
Prompt after formatting:

1. Use the following pieces of context to answer the question at the end.
2. If you don't know the answer, just say that "I am not too sure. Please refer to https://help.shopee.sg/portal/4/article/77152-Refunds-and-Return-Policy for more details" but don't make up an answer on your own.
3. If the question is completely irrelevant to Shopee's Refund Policy, you should simply reply with "The question is not related to Shopee's Refund Policy", and ask if the user would like to find out about Shopee's Refund Policy. 
For example, an irrelevant question like "How is your day?" would warrant the stated response. Additionally, if the question does not mention anything about a refund policy, it should warrant the stated response as well.

Context: Context:
content:Refunds and Return Policy

1. Application for Returns/Refunds

 

Subject to the terms and conditions in this Refunds and Return Polic